In [33]:
import requests
import json
import csv
import datetime

pos=0.5
LTC_BTC=[]
LTC_BTC_buy=[]
LTC_BTC_sell=[]

def write_csv(new_line):
    with open("common.csv", "a" ) as fout:
        writer=csv.writer(fout)
        writer.writerow((new_line))
        
fees={"trade":{'btc-e': 0.002,
               'bitfinex': 0.002,
               'poloniex': 0.0025,
               'bittrex': 0.0025,
               'kraken': 0.0026,
               'hitbtc': 0.001,
               'yobit': 0.002,
               'exmo': 0.002,
               'livecoin': 0.002,
               "cexio": 0.002},
      "trans_LTC":{'btc-e': 0.001,
                   'bitfinex': 0.001,
                   'poloniex': 0.001,
                   'bittrex': 0.002,
                   'kraken': 0.001,
                   'hitbtc': 0.001,
                   'yobit': 0.002,
                   'exmo': 0.01,
                   'livecoin': 0.001,
                   'cexio': 0.001},
      "trans_BTC":{'btc-e': 0.001,
                   'bitfinex': 0.0004,
                   'poloniex': 0.0001,
                   'bittrex': 0.001,
                   'kraken': 0.001,
                   'hitbtc': 0.00085,
                   'yobit': 0.0015,
                   'exmo': 0.001,
                   'livecoin': 0.001,
                   'cexio': 0.001}}

In [34]:
# btc-e
try:
    base_url="https://btc-e.nz/api/3/"
    pairs="ltc_btc"

    ticker_url=base_url+"ticker/"+pairs
    depth_url=base_url+"depth/"+pairs

    html_ticker=requests.get(ticker_url).text
    html_depth=requests.get(depth_url).text

    ticker_data=json.loads(html_ticker)
    depth_data=json.loads(html_depth)

    #print "Start"

    for i in ticker_data:
        q=0
        w=[]
        e=[]
        for j in depth_data[i]["bids"]:
            q=q+j[1]
            if q>pos:
                w.append(j)
            if q<pos:
                e.append(j)      
        r=0
        t=0
        for j in e:
            r=j[0]*j[1]+r
            t=j[1]+t    
        r=w[0][0]*(pos-t)+r
        bids=r/pos


        q=0
        w=[]
        e=[]
        for j in depth_data[i]["asks"]:
            q=q+j[1]
            if q>pos:
                w.append(j)
            if q<pos:
                e.append(j)    
        r=0
        t=0
        for j in e:
            r=j[0]*j[1]+r
            t=j[1]+t    
        r=w[0][0]*(pos-t)+r
        asks=r/pos    
    
        line=[str(datetime.datetime.now())[:-7], "btc-e", "LTC_BTC", str(ticker_data[i]["buy"]).replace(".",","),
              str(ticker_data[i]["sell"]).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
        
        print line 
        LTC_BTC.append(line)
        
        write_csv(line)
except:
    print "btc-e failed"
    pass    
    
# bitfinex
try:
    base_url="https://api.bitfinex.com/v1/"
    pairs=open('pairs_btc_ltc.txt').read()
    pairs=pairs.lower().replace("_","").split("\n")

    symbol_url=base_url+"symbols"
    html_symbol=requests.get(symbol_url).text
    symbol_data=json.loads(html_symbol)
    #print symbol_data

    for i in symbol_data:
        if i in pairs:
            ticker_url=base_url+"pubticker/"+i
            html_ticker=requests.get(ticker_url).text
            ticker_data=json.loads(html_ticker)        

            depth_url=base_url+"book/"+i
            html_depth=requests.get(depth_url).text
            depth_data=json.loads(html_depth)

            q=0
            w=[]
            e=[]
            for j in depth_data["bids"]:
                q=q+float(j["amount"])
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)      
            r=0
            t=0
            for j in e:
                r=float(j["price"])*float(j["amount"])+r
                t=float(j["amount"])+t    
            r=float(w[0]["price"])*(pos-t)+r
            bids=r/pos
    
            q=0
            w=[]
            e=[]
            for j in depth_data["asks"]:
                q=q+float(j["amount"])
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)    
            r=0
            t=0
            for j in e:
                r=float(j["price"])*float(j["amount"])+r
                t=float(j["amount"])+t    
            r=float(w[0]["price"])*(pos-t)+r
            asks=r/pos     
    
            line=[str(datetime.datetime.now())[:-7], "bitfinex", "LTC_BTC", str(ticker_data["ask"]).replace(".",","), 
                  str(ticker_data["bid"]).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
       
            print line 
            LTC_BTC.append(line)
        
            write_csv(line)
except:
    print "bitfinex failed"
    pass
        
# poloniex
try:
    base_url="https://poloniex.com/public?command=return"
    pairs=open('pairs_btc_ltc.txt').read().split("\n")

    ticker_url=base_url+"Ticker"
    html_ticker=requests.get(ticker_url).text
    ticker_data=json.loads(html_ticker)

    for i in ticker_data:
        if i in pairs:
            depth_url=base_url+"OrderBook&currencyPair="+i+"&depth=20"    
            html_depth=requests.get(depth_url).text
            depth_data=json.loads(html_depth)
    
            q=0
            w=[]
            e=[]
            for j in depth_data["bids"]:    
                q=q+j[1]
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)  
            r=0
            t=0
            for j in e:
                r=float(j[0])*j[1]+r
                t=j[1]+t    
            r=float(w[0][0])*(pos-t)+r
            bids=r/pos
    
            q=0
            w=[]
            e=[]
            for j in depth_data["asks"]:    
                q=q+j[1]
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)  
            r=0
            t=0
            for j in e:
                r=float(j[0])*j[1]+r
                t=j[1]+t    
            r=float(w[0][0])*(pos-t)+r
            asks=r/pos        
                
            line=[str(datetime.datetime.now())[:-7], "poloniex", "LTC_BTC", str(ticker_data[i]["lowestAsk"]).replace(".",","), 
                  str(ticker_data[i]["highestBid"]).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
        
            print line 
            LTC_BTC.append(line)
        
            write_csv(line)
except:
    print "poloniex failed"
    pass
        
# bittrex
try:
    base_url="https://bittrex.com/api/v1.1/public/"
    pairs=open('pairs_btc_ltc.txt').read()
    pairs=pairs.replace("_","-").split("\n")

    symbol_url=base_url+"getmarkets"
    html_symbol=requests.get(symbol_url).text
    symbol_data=json.loads(html_symbol)

    for i in symbol_data['result']:
        if i['MarketName'] in pairs:
            ticker_url=base_url+"getticker?market="+i['MarketName']
            html_ticker=requests.get(ticker_url).text
            ticker_data=json.loads(html_ticker)
    
            depth_url=base_url+"getorderbook?market="+i['MarketName']+"&type=both&depth=20"   
            html_depth=requests.get(depth_url).text
            depth_data=json.loads(html_depth)
    
            try:
                q=0
                w=[]
                e=[]
                for j in depth_data['result']['buy']:
                    q=q+j['Quantity']
                    if q>pos:
                        w.append(j)
                    if q<pos:
                        e.append(j) 
                r=0
                t=0
                for j in e:
                    r=j["Rate"]*j['Quantity']+r
                    t=j['Quantity']+t    
                r=w[0]["Rate"]*(pos-t)+r
                bids=r/pos
            except:
                bids=ticker_data['result']["Bid"]
        
            try:
                q=0
                w=[]
                e=[]
                for j in depth_data['result']['sell']:
                    q=q+j['Quantity']
                    if q>pos:
                        w.append(j)
                    if q<pos:
                        e.append(j) 
                r=0
                t=0
                for j in e:
                    r=j["Rate"]*j['Quantity']+r
                    t=j['Quantity']+t    
                r=w[0]["Rate"]*(pos-t)+r
                asks=r/pos
            except:
                asks=ticker_data['result']["Ask"]
        
            line = [str(datetime.datetime.now())[:-7], 'bittrex', "LTC_BTC", str(ticker_data['result']["Ask"]).replace(".",","),
                    str(ticker_data['result']["Bid"]).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
            
            print line 
            LTC_BTC.append(line)
        
            write_csv(line)
except:
    print "bittrex failed"
    pass
        
# kraken
try:
    base_url="https://api.kraken.com/0/public/"

    pairs=open('pairs_btc_ltc.txt').read()
    pairs=pairs.replace("_","").replace("BTC","XBT").split("\n")

    symbol_url=base_url+"AssetPairs"
    html_symbol=requests.get(symbol_url).text
    symbol_data=json.loads(html_symbol)

    symbols=[]
    z=0
    for i in symbol_data['result']:   
        symbols.append(symbol_data['result'][i]["altname"])    

    symbols=str(symbols).replace("[","").replace("]",'').replace("u",'').replace(" ","").replace("'","")

    ticker_url=base_url+"Ticker?pair="+symbols
    html_ticker=requests.get(ticker_url).text
    ticker_data=json.loads(html_ticker)

    for i in ticker_data['result']:
        if symbol_data['result'][i]["altname"] in pairs:
            depth_url=base_url+"Depth?pair="+symbol_data['result'][i]["altname"]+'&count=20'
            html_depth=requests.get(depth_url).text
            depth_data=json.loads(html_depth)
    
            try:
                q=0
                w=[]
                e=[]
                for j in depth_data['result'][i]['bids']:    
                    q=q+float(j[1])
                    if q>pos:
                        w.append(j)
                    if q<pos:
                        e.append(j)  
                r=0
                t=0
                for j in e:
                    r=float(j[0])*j[1]+r
                    t=j[1]+t    
                r=float(w[0][0])*(pos-t)+r
                bids=r/pos        
            except:
                bids=ticker_data['result'][i]['b'][0]        
        
            try:
                q=0
                w=[]
                e=[]
                for j in depth_data['result'][i]['asks']:    
                    q=q+float(j[1])
                    if q>pos:
                        w.append(j)
                    if q<pos:
                        e.append(j)  
                r=0
                t=0
                for j in e:
                    r=float(j[0])*j[1]+r
                    t=j[1]+t    
                r=float(w[0][0])*(pos-t)+r
                asks=r/pos        
            except:
                asks=ticker_data['result'][i]['a'][0]
        
            line=[str(datetime.datetime.now())[:-7], "kraken", "LTC_BTC", str(ticker_data['result'][i]['a'][0]).replace(".",","),
                  str(ticker_data['result'][i]['b'][0]).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
        
            print line 
            LTC_BTC.append(line)
        
            write_csv(line)
except:
    print "kraken failed"
    pass
        
# hitbtc
try:
    base_url="https://api.hitbtc.com/api/1/public/"
    pairs=open('pairs_btc_ltc.txt').read()
    pairs=pairs.replace("_","").split("\n")

    ticker_url=base_url+"ticker"
    html_ticker=requests.get(ticker_url).text
    ticker_data=json.loads(html_ticker)

    for i in ticker_data:
        if i in pairs:
            depth_url=base_url+i+"/orderbook?format_price=number&format_amount=number"
            html_depth=requests.get(depth_url).text
            depth_data=json.loads(html_depth)
    
            q=0
            w=[]
            e=[]
            for j in depth_data["bids"]:
                q=q+j[1]
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)      
            r=0
            t=0
            for j in e:
                r=j[0]*j[1]+r
                t=j[1]+t    
            r=w[0][0]*(pos-t)+r
            bids=r/pos
    
            q=0
            w=[]
            e=[]
            for j in depth_data["asks"]:
                q=q+j[1]
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)      
            r=0
            t=0
            for j in e:
                r=j[0]*j[1]+r
                t=j[1]+t    
            r=w[0][0]*(pos-t)+r
            asks=r/pos
    
            line=[str(datetime.datetime.now())[:-7], "hitbtc", "LTC_BTC", str(ticker_data[i]["ask"]).replace(".",","), 
                  str(ticker_data[i]["bid"]).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
        
            print line 
            LTC_BTC.append(line)
        
            write_csv(line)
except:
    print "hitbtc failed"
    pass
        
# yobit
try:
    base_url="https://yobit.net/api/3/"
    pairs=open('pairs_btc_ltc.txt').read().split("\n")

    symbol_url=base_url+"info"
    html_symbol=requests.get(symbol_url).text
    symbol_data=json.loads(html_symbol)

    for i in symbol_data['pairs']:
        if i.upper() in pairs:
            ticker_url=base_url+"ticker/"+i
            html_ticker=requests.get(ticker_url).text
            ticker_data=json.loads(html_ticker)
    
            depth_url=base_url+"depth/"+i+"?limit=20"
            html_depth=requests.get(depth_url).text
            depth_data=json.loads(html_depth)
    
            try:
                q=0
                w=[]
                e=[]
                for j in depth_data[i]["bids"]:
                    q=q+j[1]
                    if q>pos:
                        w.append(j)
                    if q<pos:
                        e.append(j)      
                r=0
                t=0
                for j in e:
                    r=j[0]*j[1]+r
                    t=j[1]+t    
                r=w[0][0]*(pos-t)+r
                bids=r/pos
            except:
                bids=ticker_data[i]['buy']

            try:
                q=0
                w=[]
                e=[]
                for j in depth_data[i]["asks"]:
                    q=q+j[1]
                    if q>pos:
                        w.append(j)
                    if q<pos:
                        e.append(j)    
                r=0
                t=0
                for j in e:
                    r=j[0]*j[1]+r
                    t=j[1]+t    
                r=w[0][0]*(pos-t)+r
                asks=r/pos
            except:
                asks=ticker_data[i]['sell']
    
            line=[str(datetime.datetime.now())[:-7], 'yobit', "LTC_BTC", str(ticker_data[i]['sell']).replace(".",","), 
                  str(ticker_data[i]['buy']).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
        
            print line 
            LTC_BTC.append(line)
        
            write_csv(line)
except:
    print "yobit failed"
    pass
        
# exmo
try:
    base_url="https://api.exmo.com/v1/"

    ticker_url=base_url+"ticker/"
    html_ticker=requests.get(ticker_url).text
    ticker_data=json.loads(html_ticker)

    #for i in ticker_data:
    depth_url=base_url+"order_book/?pair=LTC_BTC&limit=20"
    html_depth=requests.get(depth_url).text
    depth_data=json.loads(html_depth)
    
    try:
        q=0
        w=[]
        e=[]
        for j in depth_data["LTC_BTC"]['bid']:    
            q=q+float(j[1])
            if q>pos:
                w.append(j)
            if q<pos:
                e.append(j)  
        r=0
        t=0
        for j in e:
            r=float(j[0])*float(j[1])+r
            t=float(j[1])+t    
        r=float(w[0][0])*(pos-t)+r
        bids=r/pos
    except:
        bids=ticker_data["LTC_BTC"]['buy_price']
        
    try:
        q=0
        w=[]
        e=[]
        for j in depth_data["LTC_BTC"]['ask']:    
            q=q+float(j[1])
            if q>pos:
                w.append(j)
            if q<pos:
                e.append(j)  
        r=0
        t=0
        for j in e:
            r=float(j[0])*float(j[1])+r
            t=float(j[1])+t    
        r=float(w[0][0])*(pos-t)+r
        asks=r/pos
    except:
        asks=ticker_data["LTC_BTC"]['sell_price']
        
    line=[str(datetime.datetime.now())[:-7], "exmo", "LTC_BTC", str(ticker_data["LTC_BTC"]['sell_price']).replace(".",","), 
          str(ticker_data["LTC_BTC"]['buy_price']).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
    
    print line 
    LTC_BTC.append(line)

    write_csv(line)
except:
    print "exmo failed"
    pass

#livecoin
try:
    base_url="https://api.livecoin.net/exchange/"

    ticker_url=base_url+"ticker?currencyPair=LTC/BTC"
    depth_url=base_url+"order_book?currencyPair=LTC/BTC"

    html_ticker=requests.get(ticker_url).text
    html_depth=requests.get(depth_url).text

    ticker_data=json.loads(html_ticker)
    depth_data=json.loads(html_depth)

    #for i in ticker_data:
    try:
        q=0
        w=[]
        e=[]
        for j in depth_data["bids"]:
            q=q+float(j[1])
            if q>pos:
                w.append(j)
            if q<pos:
                e.append(j)     
            
        r=0
        t=0
        for j in e:
            r=float(j[0])*float(j[1])+r
            t=float(j[1])+t    
        r=float(w[0][0])*(pos-t)+r
        bids=r/pos 
    
    except:
        bids=ticker_data["best_bid"]
        
    try:
        q=0
        w=[]
        e=[]
        for j in depth_data["asks"]:
            q=q+float(j[1])
            if q>pos:
                w.append(j)
            if q<pos:
                e.append(j)     
            
        r=0
        t=0
        for j in e:
            r=float(j[0])*float(j[1])+r
            t=float(j[1])+t    
        r=float(w[0][0])*(pos-t)+r
        asks=r/pos      
    
    except:
        asks=ticker_data["best_ask"]    
        
        
    line = [str(datetime.datetime.now())[:-7], "livecoin", "LTC_BTC", str(ticker_data["best_ask"]).replace(".",","), 
            str(ticker_data["best_bid"]).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
    
    print line 
    LTC_BTC.append(line)
    
    write_csv(line)
    
except:
    print "livecoin failed"
    pass

for i in LTC_BTC:     
    trans_comis_buy = fees["trans_LTC"][i[1]]*(float(str(i[5]).replace(",","."))+float(str(i[6]).replace(",",".")))/2
    trade_comis_buy = float(str(i[5]).replace(",","."))*fees["trade"][i[1]]*pos
    LTC_BTC_buy.append([float(str(i[5]).replace(",","."))+trade_comis_buy+trans_comis_buy, i[1]])
    
    trans_comis_sell = fees["trans_BTC"][i[1]]
    trade_comis_sell = float(str(i[6]).replace(",","."))*fees["trade"][i[1]]*pos               
    LTC_BTC_sell.append([float(str(i[6]).replace(",","."))-trade_comis_sell-trans_comis_sell, i[1]])


new_line=[str(datetime.datetime.now())[:-7], "LTC_BTC", min(zip(*zip(*LTC_BTC_buy)[0:2]))[1], 
         str(min(zip(*zip(*LTC_BTC_buy)[0:2]))[0]).replace(".",","),
         str(max(zip(*zip(*LTC_BTC_sell)[0:2]))[0]).replace(".",","), 
         max(zip(*zip(*LTC_BTC_sell)[0:2]))[1],
         str((max(zip(*zip(*LTC_BTC_sell)[0:2]))[0]/min(zip(*zip(*LTC_BTC_buy)[0:2]))[0]-1)*100).replace(".",",")+"%"]

print new_line
write_csv(new_line)

['2017-06-22 13:39:57', 'btc-e', 'LTC_BTC', '0,01715', '0,0171', '0,01715', '0,0171']
['2017-06-22 13:39:58', 'bitfinex', 'LTC_BTC', '0,017178', '0,017144', '0,017178', '0,017144']
['2017-06-22 13:39:59', 'poloniex', 'LTC_BTC', '0,01720008', '0,01718450', '0,01719663', '0,0171845']
['2017-06-22 13:40:00', 'bittrex', 'LTC_BTC', '0,0171999', '0,01719629', '0,0171999', '0,01719629']
['2017-06-22 13:40:03', 'kraken', 'LTC_BTC', '0,01720200', '0,01713900', '0,01720200', '0,017139']
['2017-06-22 13:40:03', 'hitbtc', 'LTC_BTC', '0,01717', '0,01715', '0,017174', '0,01716']
['2017-06-22 13:40:04', 'yobit', 'LTC_BTC', '0,01746', '0,01730001', '0,01746', '0,01730001']
['2017-06-22 13:40:04', 'exmo', 'LTC_BTC', '0,01715', '0,01705762', '0,017164064', '0,0170530536224']
['2017-06-22 13:40:06', 'livecoin', 'LTC_BTC', '0,01715', '0,01699999', '0,017190000005', '0,01699999']
['2017-06-22 13:40:06', 'LTC_BTC', 'btc-e', '0,017184275', '0,017063019375', 'poloniex', '-0,705619672637%']
